# Transforming Data with Joins & Aggregations

In [0]:
# Import the necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, avg, round, countDistinct

In [0]:
display(dbutils.fs.ls("abfss://etl1@dbstoragebbpbs73u57xmm.dfs.core.windows.net/Exercise2/"))
# Load our streaming events dataset
events_path = "abfss://etl1@dbstoragebbpbs73u57xmm.dfs.core.windows.net/Exercise2/streaming_events.csv"
events_df = spark.read.option("header", "true").option("inferSchema", "true").csv(events_path)

# Load our reference datasets
users_path = "abfss://etl1@dbstoragebbpbs73u57xmm.dfs.core.windows.net/Exercise2/users.csv"
users_df = spark.read.option("header", "true").option("inferSchema", "true").csv(users_path)

content_path = "abfss://etl1@dbstoragebbpbs73u57xmm.dfs.core.windows.net/Exercise2/content_catalog.csv"
content_df = spark.read.option("header", "true").option("inferSchema", "true").csv(content_path)

In [0]:
# Look at the schema of our datasets to understand the join keys
print("Events Schema (preview):")
events_df.printSchema()

print("\nUsers Schema (preview):")
users_df.printSchema()

print("\nContent Schema (preview):")
content_df.printSchema()

### Joins

In [0]:
# Basic INNER JOIN to bring user information to our events
events_with_users = events_df.join(
    users_df,
    on="user_id",
    how="inner"
)

print("Events joined with Users:")
events_with_users.select("event_id", "user_id", "subscription_tier", "age_group").limit(3).display()

# Check how many events we have before and after the join
print(f"Events before join: {events_df.count()}")
print(f"Events after inner join with users: {events_with_users.count()}")

In [0]:
# LEFT JOIN to keep all events, even if the user isn't in our users table
events_with_users_left = events_df.join(
    users_df,
    on="user_id",
    how="left"
)

print(f"Events after left join with users: {events_with_users_left.count()}")

# Join with the content catalog to get content details
events_enriched = events_with_users_left.join(
    content_df,
    on="content_id",
    how="left"
)

print("Fully enriched events:")
events_enriched.select("event_id", "user_id", "subscription_tier", "content_id", "title", "genre").limit(3).display()


### Aggregations

In [0]:
# Basic aggregation: Count events by subscription tier
events_by_tier = events_enriched.groupBy("subscription_tier") \
    .agg(count("*").alias("event_count")) \
    .orderBy(col("event_count").desc())

print("Events by subscription tier:")
events_by_tier.display()

In [0]:
# Average streaming duration by content genre
avg_duration_by_genre = events_enriched.groupBy("genre") \
    .agg(
        count("*").alias("event_count"),
        round(avg("duration_seconds"), 2).alias("avg_duration_seconds")
    ) \
    .orderBy(col("avg_duration_seconds").desc())

print("Average streaming duration by content genre:")
avg_duration_by_genre.limit(5).display()

In [0]:
# Complex aggregation: Content performance by subscription tier and genre
content_performance = events_enriched.groupBy("subscription_tier", "genre") \
    .agg(
        countDistinct("user_id").alias("unique_viewers"),
        round(avg("duration_seconds") / 60, 2).alias("avg_minutes_per_view"),
        round(sum("duration_seconds") / 3600, 2).alias("total_hours_watched")
    ) \
    .orderBy(col("subscription_tier"), col("total_hours_watched").desc())

print("Content performance by subscription tier and genre:")
content_performance.limit(5).display()

In [0]:
# Save our enriched dataset for downstream analytics
output_path = "pyspark/video-streaming-data/module3-transform/joins_aggregations/enriched_events"

# Save as Parquet (columnar format)
events_enriched.write.mode("overwrite").parquet(output_path)

print(f"Enriched data saved to {output_path}")